In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
# ! mkdir -p ~/.kaggle/
# ! mv kaggle.json ~/.kaggle/

In [3]:
path = Config.data_path()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('/home/siddhant/.fastai/data/planet')

In [18]:
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
# ! unzip -q -n {path}/train_v2.csv.zip -d {path}

/bin/sh: 1: kaggle: not found
/bin/sh: 1: kaggle: not found
unzip:  cannot find or open /home/siddhant/.fastai/data/planet/train_v2.csv.zip, /home/siddhant/.fastai/data/planet/train_v2.csv.zip.zip or /home/siddhant/.fastai/data/planet/train_v2.csv.zip.ZIP.


/bin/sh: 1: conda: not found
